#### Dataframe Joins and Optimizations
SparkSQL builds in several optimizations while performing joins and other operations on dataframes.

We can check out the execution plan of SparkSQL for any operation using the function "explain()"

When one of the dataframes in the join operation is small enough to fit into a workernode then Spark SQL plans to "broadcast" this dataframe. This is generally referred to as BroadcashJoin. When a dataframe is "broadcasted" then a copy of it is sent to all worker nodes. This avoids shuffling of data when joining it with the larger dataframe as the smaller dataframe is compltely available locally for each partition of the larger dataframe.

The above optimization is done automatically by SparkSQL's built-in optimizer. However we can also check to ensure it is done and in case it is not done it can be done manually also as shown in the example below.

The setting "spark.sql.autoBroadcastJoinThreshold" determines whether a dataframe is small enough to be broadcast or not. The default of this setting is 10 MB (10485760). We can override this when we know that the smaller dataframe is just above this threshold and can be managed to be fit into each worker node. 

Import and initiate findspark to begin with.

In [ ]:
# The following two lines are required if you are running PySpark using Jupyter notebook with Python3 as kernel 
import findspark
findspark.init("/usr/local/spark")

Now import pyspark. And start SparkSession

In [ ]:
# The following three lines are required if you are running PySpark using Jupyter notebook with Python3 as kernel 
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL Broadcast Join example").getOrCreate()

Create DataFrame from data source - csv file

In [ ]:
customerDF = spark.read.load("customers.tsv", format="csv", sep="\t", inferSchema="true")
salestxnDF = spark.read.load("salestxns.tsv", format="csv", sep="\t", inferSchema="true")

Different types of operations on DataFrames

In [ ]:
customerDF.printSchema()

In [ ]:
salestxnDF.printSchema()

In [ ]:
new_names_cDF = ['customer_id','customer_name','customer_city','customer_state','customer_zipcode']

In [ ]:
new_names_sDF = ['salestxn_id','category_id','category_name','product_id','product_name','product_price',\
              'product_quantity','customer_id']

In [ ]:
cDF2=customerDF.toDF(*new_names_cDF)

In [ ]:
sDF2=salestxnDF.toDF(*new_names_sDF)

In [ ]:
cDF2.printSchema()

In [ ]:
cDF2.show(5)

In [ ]:
sDF2.printSchema()

In [ ]:
sDF2.show(5)

Join the two dataframes on customer_id column

In [ ]:
jDF=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [ ]:
jDF.printSchema()

In [ ]:
jDF.show(5)

Use explain() function to see the type of join used by Spark SQL

In [ ]:
jDF.explain()

Using explain(True) function gives the outline of the initial logical plans besides the final physical plan used for executing the join operation

In [ ]:
jDF.explain(True)

The function explain() can be used on the dataframe and also on the join operation.

Note that when we use explain() function on the join operaton the join is not performed. Only the plan is explained.

In [ ]:
jDF=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id).explain()

In [ ]:
type(jDF)

In [ ]:
# This line gives an error as jDF dataframe is not created since explain() function does not perform the join function.
# It only explains the plan. 
jDF.printSchema()

In [ ]:
# This line gives an error as jDF dataframe is not created since explain() function does not perform the join function.
# It only explains the plan. 
jDF.show(5)

In [ ]:
jDF=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id).explain(True)

In [ ]:
type(jDF)

In [ ]:
# This line gives an error as jDF dataframe is not created since explain() function does not perform the join function.
# It only explains the plan. 
jDF.printSchema()

In [ ]:
# This line gives an error as jDF dataframe is not created since explain() function does not perform the join function.
# It only explains the plan. 
jDF.show(5)

In [ ]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

By setting this value to -1 broadcasting can be disabled.

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

In [ ]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

In [ ]:
jDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [ ]:
# Since autoBroadcastJoinThreshold is disabled by setting it to -1, physical plan shows BroadCastHashJoin it is not used.
jDF1.explain()

In [ ]:
# Since autoBroadcastJoinThreshold is disabled by setting it to -1, physical plan shows BroadCastHashJoin it is not used.
jDF1.explain(True)

We can notice that broadcast join is not used now as the threshold is disabled.

In [ ]:
jDF1.printSchema()

In [ ]:
jDF1.show(5)

Using the join operation by interchanging the dataframes from left to right.

In [ ]:
jDF2=sDF2.join(cDF2,sDF2.customer_id==cDF2.customer_id)

In [ ]:
jDF2.explain()

In [ ]:
jDF2.explain(True)

In [ ]:
jDF2.printSchema()

In [ ]:
jDF2.show(5)

In [ ]:
from pyspark.sql import functions as F

The lines below show how to mark a dataframe for broadcast in case required to be done manually.

In [ ]:
jDF3=sDF2.join(cDF2.hint("broadcast"),sDF2.customer_id==cDF2.customer_id)

In [ ]:
# Since we added the hint("broadcast") the plan uses BroadcastHashJoin 
# though the autoBroadcastJoinThreshold is disabled by setting it to -1
jDF3.explain()

In [ ]:
# Since we added the hint("broadcast") the plan uses BroadcastHashJoin 
# though the autoBroadcastJoinThreshold is disabled by setting it to -1
jDF3.explain(True)

In [ ]:
jDF3.printSchema()

In [ ]:
jDF3.show(5)

In [ ]:
jDF4=sDF2.join(F.broadcast(cDF2),sDF2.customer_id==cDF2.customer_id)
type(jDF4)

In [ ]:
# Since we gave the F.broadcast(DataFrame) to specify broadcast join to be used for the dataframe
# the plan uses BroadcastHashJoin though the autoBroadcastJoinThreshold is disabled by setting it to -1
jDF4.explain()

In [ ]:
# Since we gave the F.broadcast(DataFrame) to specify broadcast join to be used for the dataframe
# the plan uses BroadcastHashJoin though the autoBroadcastJoinThreshold is disabled by setting it to -1
jDF4.explain(True)

In [ ]:
jDF4.printSchema()

In [ ]:
jDF4.show(5)

In [ ]:
# Another way (syntax) of specifying broadcast join for a dataframe
jDF5=cDF2.hint("broadcast").join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [ ]:
jDF5.explain()

In [ ]:
jDF5.explain(True)

In [ ]:
jDF5.printSchema()

In [ ]:
jDF5.show(5)

In [ ]:
jDF6=F.broadcast(cDF2).join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [ ]:
jDF6.explain()

In [ ]:
jDF6.explain(True)

In [ ]:
jDF6.printSchema()

In [ ]:
jDF6.show(5)